Mammal Habitats Under Threat	bio.012.2	http://sedac.ciesin.columbia.edu/data/set/species-global-mammal-richness-2015

In [19]:
### Not sure why this is bigger than the Amphibian data... all signs point to identical methods

import boto3

import rasterio
import numpy as np

import os
import sys
import threading

Establish local and s3 file locations

In [35]:
# Local storage
local_folder = "/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/"
rw_data_type = local_folder + "Biodiversity/"

local_orig = rw_data_type + "mammals_all_threats.tif"
local_edit = local_orig[:-4] + "_edit.tif"

# S3 storage
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/bio_012_2_mammalian_species_under_threat/"

s3_key_orig = s3_folder + "bio_012_2_mammalian_species_under_threat.tif"
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [36]:
s3_download.meta.client.download_file(s3_bucket, s3_key_orig, local_orig)

<b>Regardless of any needed edits, upload original file</b>

<i>Upload tif to S3 folder</i>

http://boto3.readthedocs.io/en/latest/guide/s3-example-creating-buckets.html

<i>Monitor Progress of Upload</i>

http://boto3.readthedocs.io/en/latest/_modules/boto3/s3/transfer.html
https://boto3.readthedocs.io/en/latest/guide/s3.html#using-the-transfer-manager

In [14]:
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

Create edit file

In [37]:
# Adjust kwargs

with rasterio.open(local_orig) as src:
    kwargs = src.profile

print(kwargs)

nodata_val = 0
data_type = rasterio.int32
t_crs = 'EPSG:4326'

kwargs.update(
    old_nodata = kwargs["nodata"],
    driver='GTiff',
    dtype=data_type,  #rasterio.int16, rasterio.int32, rasterio.uint8,rasterio.uint16, rasterio.uint32, rasterio.float32, rasterio.float64
    count=1,
    compress='lzw',
    nodata=nodata_val,
    bigtiff='NO',
    crs = t_crs,
    tiled = False
)

kwargs.pop("blockxsize", None)
kwargs.pop("blockysize", None)

print(kwargs)

{'driver': 'GTiff', 'dtype': 'int32', 'nodata': 2147483647.0, 'width': 43200, 'height': 21600, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.00833330000000387, 0.0, 89.99928000008359, 0.0, -0.00833330000000387), 'affine': Affine(0.00833330000000387, 0.0, -180.0,
       0.0, -0.00833330000000387, 89.99928000008359), 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'int32', 'nodata': 0, 'width': 43200, 'height': 21600, 'count': 1, 'crs': 'EPSG:4326', 'transform': (-180.0, 0.00833330000000387, 0.0, 89.99928000008359, 0.0, -0.00833330000000387), 'affine': Affine(0.00833330000000387, 0.0, -180.0,
       0.0, -0.00833330000000387, 89.99928000008359), 'tiled': False, 'interleave': 'band', 'old_nodata': 2147483647.0, 'compress': 'lzw', 'bigtiff': 'NO'}


In [38]:
with rasterio.open(local_orig) as src:
    windows = src.block_windows()
    with rasterio.open(local_edit, 'w', **kwargs) as dst:
        for win_id, window in windows:
            src_data = src.read(1, window=window).astype(kwargs["dtype"])
            np.putmask(src_data, src_data==kwargs["old_nodata"], kwargs["nodata"])
            dst.write_band(1, src_data, window=window)

/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [28]:
with rasterio.open(local_edit) as src:
    data = src.read(1)
    
np.unique(data, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], dtype=int32),
 array([169243367, 154184574,  53840830, 200040897, 263286993,  57000855,
         18392252,   6902183,   2093180,   2953281,   1290368,    607972,
           383123,    381171,    352998,    355560,    289393,    224771,
           184635,    223244,    242506,    188104,    111229,     88250,
            76795,     52540,     45673,     36178,     21961,     14899,
             7136,      1993,      1054,        35]))

Upload edited files to S3

In [39]:
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
           Callback=ProgressPercentage(local_edit))

/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/mammals_all_threats_edit.tif  84103091 / 84103091.0  (100.00%)